System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin21.5.0)
  CPU: 10 × Apple M1 Max
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, apple-m1)
  Threads: 1 on 8 virtual cores


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Downloads/SSS/Courses/Bio257_23Spring/HW/hw3`


Status `~/Downloads/SSS/Courses/Bio257_23Spring/HW/hw3/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [31c24e10] Distributions v0.25.89
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random


In [3]:
using LinearAlgebra, Random
using BenchmarkTools, Distributions

[ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
[ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]


Consider a linear mixed effects model
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

## Q1 Formula (10 pts)

Write down the log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ given parameters $(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$. 

## Sol Q1

Notice that the sum of two independent normal random variables is also a normal random variable. Hence, $Z_{i}\gamma \sim N(0_{n_{i}}, Z_{i}\Sigma Z_{i}^{T})$, $Z_{i}\gamma + \epsilon_{i} \sim N(0_{n_{i}}, Z_{i}\Sigma Z_{i}^{T} + \sigma^{2}I_{n_{i}})$, and $Y_{i} \sim N(X_{i}\beta, Z_{i}\Sigma Z_{i}^{T} + \sigma^{2}I_{n_{i}})$.

The likelihood of the $i$th datum proceeds:

\begin{eqnarray*}
f(y_{i} \vert \beta,\Sigma, \sigma^{2}) &=& -\frac{1}{(2\pi)^{n_{i}/2}\lvert Z_{i}\Sigma Z_{i}^{T} + \sigma^{2}I_{n_{i}}\rvert^{1/2}}\exp\left(-\frac{1}{2}(y_{i} - X_{i}\beta)^{T}(Z_{i}\Sigma Z_{i}^{T} + \sigma^{2}I_{n_{i}})^{-1}(y_{i} - X_{i}\beta)\right)
\end{eqnarray*}

The corresponding log-likelihood would be acquired by taking the log of the likelihood:

\begin{eqnarray*}
l(\beta, \Sigma, \sigma^{2}\vert y_{i}) &=& -\frac{n_{i}}{2}\log(2\pi) - \frac{1}{2}\log \lvert Z_{i}\Sigma Z_{i}^{T} + \sigma^{2}I_{n_{i}}\rvert - \frac{1}{2}(y_{i} - X_{i}\beta)^{T}(Z_{i}\Sigma Z_{i}^{T} + \sigma^{2}I_{n_{i}})^{-1}(y_{i} - X_{i}\beta)
\end{eqnarray*}

## Q2 Start-up code

Use the following template to define a type `LmmObs` that holds an LMM datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$. 

In [4]:
# define a type that holds LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y :: Vector{T}
    X :: Matrix{T}
    Z :: Matrix{T}
    # working arrays
    # whatever intermediate vectors/arrays you may want to pre-allocate
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    yty        :: T
    xty        :: Vector{T}
    zty        :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    storage_qq :: Matrix{T}
end

# constructor
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
        ) where T <: AbstractFloat
    storage_p  = Vector{T}(undef, size(X, 2))
    storage_q  = Vector{T}(undef, size(Z, 2))
    yty        = dot(y, y)
    xty        = transpose(X) * y
    zty        = transpose(Z) * y
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    storage_qq = similar(ztz)
    LmmObs(y, X, Z, storage_p, storage_q, yty, xty, zty, xtx, ztx, ztz, storage_qq)
end

LmmObs

Write a function, with interface   
```julia
logl!(obs, β, L, σ²)
```
that evaluates the log-likelihood of the $i$-th datum. Here `L` is the lower triangular Cholesky factor from the Cholesky decomposition `Σ=LL'`. Make your code efficient in the $n_i \gg q$ case. Think the intensive longitudinal measurement setting.  

In [5]:
using LinearAlgebra
import LinearAlgebra: BlasReal, copytri!

function logl!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        L   :: Matrix{T}, 
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)    
    # compute and return the log-likelihood
    # starting constants
    l = -(n * (log(2) + log(pi)) + (n - q) * log(σ²)) / 2
#    l -= n / 2 * log(2 * pi) + n / 2 * log(σ²)
    
    # compute V = σ²I_q + L'Zᵢ'ZᵢL to compute determinant.
    copy!(obs.storage_qq, obs.ztz)
    BLAS.trmm!('L', 'L', 'T', 'N', one(T), L, obs.storage_qq)
    BLAS.trmm!('R', 'L', 'N', 'N', one(T), L, obs.storage_qq)
#    obs.storage_qq .+= s2I_q
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ²
    end
    #note: numerical precision issues were fixed after the same matrix was used
    # to compute the determinant.
    # Use diagonal elements of cholesky factorization
    #note: computing logdet allocates. Don't need LU on this. 
    LAPACK.potrf!('L', obs.storage_qq)
#    qr!(obs.storage_qq)
    @inbounds for j in 1:q
        l -= log(obs.storage_qq[j, j])
    end

    # note: Don't invert the matrix. Just do triangular (or symmetric?) solve.
    
    #add quadratic terms
    copy!(obs.storage_q, obs.zty)
    mul!(obs.storage_q, obs.ztx, β, -one(T), one(T))
    quad = obs.yty + dot(β, obs.xtx, β) - 2 * dot(obs.xty, β)
    
    # get L'Z'(yᵢ - Xᵢβ)
    BLAS.trmv!('L', 'T', 'N', L, obs.storage_q) # TODO: gemv! instead?
    # Calling V = σ²I + L'Zᵢ'ZᵢL, let L_V be the lower triangular matrix
    # from the Cholesky of V
    # and get (L_V)⁻¹L'Z'(yᵢ - Xᵢβ)
    BLAS.trsv!('L', 'N', 'N', obs.storage_qq, obs.storage_q)

    # aggregate quadratic terms for (yᵢ - Xᵢβ)'ZL(L_V)⁻ᵀ(L_V)⁻¹L'Z'(yᵢ - Xᵢβ)
    quad -= dot(obs.storage_q, obs.storage_q)
    l -= quad / (2 * σ²)
    
#    sleep(1e-3) # wait 1 ms as if your code takes 1ms
    return l
end

logl! (generic function with 1 method)

**Hint**: This function shouldn't be very long. Mine, obeying 92-character rule, is 30 lines. If you find yourself writing very long code, you're on the wrong track. Think about algorithm (flop count) first then use BLAS functions to reduce memory allocations.

## Q3 Correctness (15 pts)

Compare your result (both accuracy and timing) to the [Distributions.jl](https://juliastats.org/Distributions.jl/stable/multivariate/#Distributions.AbstractMvNormal) package using following data.

In [6]:
Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form an LmmObs object
obs = LmmObs(y, X, Z)

LmmObs{Float64}([-1.450910909560209, 1.5185224894450862, 5.265021705624027, 4.485272594164557, 0.694969966642933, 1.7723256696372405, 1.1065838446466518, 3.729166811829607, 4.288899999400642, 2.8241842645202406  …  4.058027151891634, 1.0909724390970443, 0.026692243086209766, -0.8927757653299448, 6.94725248926293, 3.5193020855673436, 4.914007299083773, 2.1610206566690797, 1.857389542694909, 6.513818951020866], [1.0 0.6790633442371218 … 0.5400611947971554 -0.632040682052606; 1.0 1.2456776800889142 … -0.4818455756130373 0.6467830314674976; … ; 1.0 0.0733124748775436 … 0.6125080259511859 0.4181258283983667; 1.0 -1.336609049786048 … -0.18567490803712938 1.0745977099307227], [1.0 -1.0193326822839996 -0.15855601254314888; 1.0 1.7462667837699666 -0.4584376230657152; … ; 1.0 1.4843185594903878 0.42458303115266854; 1.0 0.3791714762820068 0.25150666970865837], [2.388801279e-314, 2.388090623e-314, 2.164590566e-314, 2.2432432237e-314, 0.0], [5.0e-324, 5.0e-324, 5.0e-324], 20378.320552515157, [4209.

This is the standard way to evaluate log-density of a multivariate normal, using the Distributions.jl package. Let's evaluate the log-likelihood of this datum.

In [7]:
μ  = X * β
Ω  = Z * Σ * transpose(Z) +  σ² * I
mvn = MvNormal(μ, Symmetric(Ω)) # MVN(μ, Σ)
logpdf(mvn, y)

-3256.179335805832

Check that your answer matches that from Distributions.jl

In [8]:
L = Matrix(cholesky(Σ).L)
logl!(obs, β, L, σ²)

-3256.1793358058308

**You will lose all 15 + 30 + 30 = 75 points** if the following statement throws `AssertionError`.

In [9]:
@assert logl!(obs, β, Matrix(cholesky(Σ).L), σ²) ≈ logpdf(mvn, y)

## Q4 Efficiency (30 pts)

Benchmarking your code and compare to the Distributions.jl function `logpdf`.

In [10]:
# benchmark the `logpdf` function in Distribution.jl
bm1 = @benchmark logpdf($mvn, $y)

BenchmarkTools.Trial: 8658 samples with 1 evaluation.
 Range (min … max):  557.833 μs …  6.040 ms  ┊ GC (min … max): 0.00% … 90.38%
 Time  (median):     569.959 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   573.262 μs ± 59.532 μs  ┊ GC (mean ± σ):  0.11% ±  0.97%

           ▃▅▇█▆▅▃▁                                             
  ▁▁▁▁▂▃▄▇█████████▇▆▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  558 μs          Histogram: frequency by time          612 μs <

 Memory estimate: 31.52 KiB, allocs estimate: 3.

In [11]:
# benchmark your implementation
L = Matrix(cholesky(Σ).L)
bm2 = @benchmark logl!($obs, $β, $L, $σ²)

BenchmarkTools.Trial: 10000 samples with 200 evaluations.
 Range (min … max):  401.250 ns …  1.434 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     414.790 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   418.304 ns ± 16.845 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

         ▄▅█▆▁                                                  
  ▁▁▁▁▂▄▇██████▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  401 ns          Histogram: frequency by time          483 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

The points you will get is
$$
\frac{x}{1000} \times 30,
$$
where $x$ is the speedup of your program against the standard method.

In [12]:
# this is the points you'll get
clamp(median(bm1).time / median(bm2).time / 1000 * 30, 0, 30)

30.0

**Hint**: Apparently I am using 1000 as denominator because I expect your code to be at least $1000 \times$ faster than the standard method.

## Q5 Memory (30 pts)

You want to avoid memory allocation in the "hot" function `logl!`. You will lose 1 point for each `1 KiB = 1024 bytes` memory allocation. In other words, the points you get for this question is

In [13]:
clamp(30 - median(bm2).memory / 1024, 0, 30)

30.0

**Hint**: I am able to reduce the memory allocation to 0 bytes.

## Q6 Misc (15 pts)

Coding style, Git workflow, etc. For reproducibity, make sure we (TA and myself) can run your Jupyter Notebook. That is how we grade Q4 and Q5. If we cannot run it, you will get zero points.